(use/api)=

# Python API

## Initialisation

In [1]:
from pathlib import Path
import nbformat as nbf
from jupyter_cache import get_cache
from jupyter_cache.base import NbBundleIn
from jupyter_cache.executors import load_executor
from jupyter_cache.utils import (
    tabulate_cache_records, 
    tabulate_stage_records
)

In [2]:
cache = get_cache(".jupyter_cache")
cache.clear_cache()
cache

JupyterCacheBase('/Users/cjs14/GitHub/jupyter-cache/docs/using/.jupyter_cache')

In [3]:
print(cache.list_cache_records())
print(cache.list_staged_records())

[]
[]


## Cacheing Notebooks

In [4]:
record = cache.cache_notebook_file(
    path=Path("example_nbs", "basic.ipynb")
)
record

NbCacheRecord(pk=1)

In [5]:
record.to_dict()

{'hashkey': '818f3412b998fcf4fe9ca3cca11a3fc3',
 'description': '',
 'created': datetime.datetime(2020, 3, 12, 23, 57, 4, 973366),
 'data': {},
 'pk': 1,
 'uri': 'example_nbs/basic.ipynb',
 'accessed': datetime.datetime(2020, 3, 12, 23, 57, 4, 973385)}

In [6]:
cache.list_cache_records()

[NbCacheRecord(pk=1)]

In [7]:
cache.get_cache_record(1)

NbCacheRecord(pk=1)

In [8]:
nb_bundle = cache.get_cache_bundle(1)
nb_bundle

NbBundleOut(nb=Notebook(cells=1), record=NbCacheRecord(pk=1), artifacts=NbArtifacts(paths=0))

In [9]:
nb_bundle.nb

{'cells': [{'cell_type': 'code',
   'execution_count': 1,
   'metadata': {},
   'outputs': [{'name': 'stdout', 'output_type': 'stream', 'text': '1\n'}],
   'source': 'a=1\nprint(a)'}],
 'metadata': {'kernelspec': {'display_name': 'Python 3',
   'language': 'python',
   'name': 'python3'},
  'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
   'file_extension': '.py',
   'mimetype': 'text/x-python',
   'name': 'python',
   'nbconvert_exporter': 'python',
   'pygments_lexer': 'ipython3',
   'version': '3.6.1'},
  'test_name': 'notebook1'},
 'nbformat': 4,
 'nbformat_minor': 2}

In [10]:
record = cache.cache_notebook_file(
    path=Path("example_nbs", "basic.ipynb")
)

CachingError: Notebook already exists in cache and overwrite=False.

In [ ]:
notebook = nbf.read(str(Path("example_nbs", "basic.ipynb")), 4)
notebook

In [ ]:
cache.match_cache_notebook(notebook)

In [ ]:
notebook.cells[0].source = "change some text"

In [14]:
cache.match_cache_notebook(notebook)

NbCacheRecord(pk=1)

In [15]:
notebook.cells[1].source = "change some source code"

In [16]:
cache.match_cache_notebook(notebook)

KeyError: 'Cache record not found for NB with hashkey: 74933d8a93d1df9caad87b2e6efcdc69'

In [ ]:
print(cache.diff_nbnode_with_cache(1, notebook, as_str=True))

In [ ]:
nb_bundle = NbBundleIn(
    nb=notebook,
    uri=Path("example_nbs", "basic.ipynb"),
    data={"tag": "mytag"}
)
cache.cache_notebook_bundle(nb_bundle)

In [ ]:
print(tabulate_cache_records(
    cache.list_cache_records(), path_length=1, hashkeys=True
))

## Staging Notebooks for Execution

In [20]:
record = cache.stage_notebook_file(Path("example_nbs", "basic.ipynb"))
record

NbStageRecord(pk=1)

In [21]:
record.to_dict()

{'assets': [],
 'traceback': '',
 'pk': 1,
 'uri': '/Users/cjs14/GitHub/jupyter-cache/docs/using/example_nbs/basic.ipynb',
 'created': datetime.datetime(2020, 3, 12, 23, 57, 12, 376058)}

In [22]:
print(tabulate_stage_records(
    cache.list_staged_records(), path_length=2, cache=cache
))

ID  URI                      Created             Assets    Cache ID
----  -----------------------  ----------------  --------  ----------
   1  example_nbs/basic.ipynb  2020-03-12 23:57         0           1


In [23]:
cache.get_cache_record_of_staged(1)

NbCacheRecord(pk=1)

In [24]:
record = cache.stage_notebook_file(Path("example_nbs", "basic_failing.ipynb"))

In [25]:
cache.list_staged_unexecuted()

[NbStageRecord(pk=2)]

In [26]:
cache.discard_staged_notebook(1)

In [27]:
print(tabulate_stage_records(
    cache.list_staged_records(), path_length=2, cache=cache
))

ID  URI                              Created             Assets
----  -------------------------------  ----------------  --------
   2  example_nbs/basic_failing.ipynb  2020-03-12 23:57         0


## Execution

In [36]:
cache.clear_cache()
cache.stage_notebook_file(Path("example_nbs", "basic.ipynb"))
cache.stage_notebook_file(Path("example_nbs", "basic_failing.ipynb"))

NbStageRecord(pk=2)

In [37]:
from logging import basicConfig, INFO
basicConfig(level=INFO)

executor = load_executor("basic", cache=cache)
executor

JupyterExecutorBasic(cache=JupyterCacheBase('/Users/cjs14/GitHub/jupyter-cache/docs/using/.jupyter_cache'))

In [38]:
result = executor.run_and_cache()
result

INFO:jupyter_cache.executors.base:Executing: /Users/cjs14/GitHub/jupyter-cache/docs/using/example_nbs/basic.ipynb
INFO:jupyter_cache.executors.base:Execution Succeeded: /Users/cjs14/GitHub/jupyter-cache/docs/using/example_nbs/basic.ipynb
INFO:jupyter_cache.executors.base:Executing: /Users/cjs14/GitHub/jupyter-cache/docs/using/example_nbs/basic_failing.ipynb
ERROR:jupyter_cache.executors.base:Execution Failed: /Users/cjs14/GitHub/jupyter-cache/docs/using/example_nbs/basic_failing.ipynb


{'succeeded': ['/Users/cjs14/GitHub/jupyter-cache/docs/using/example_nbs/basic.ipynb'],
 'excepted': ['/Users/cjs14/GitHub/jupyter-cache/docs/using/example_nbs/basic_failing.ipynb'],
 'errored': []}

In [41]:
record = cache.get_cache_record(1)
record.to_dict()

{'hashkey': '818f3412b998fcf4fe9ca3cca11a3fc3',
 'description': '',
 'created': datetime.datetime(2020, 3, 13, 0, 2, 58, 780059),
 'data': {'execution_seconds': 1.492067899999995},
 'pk': 1,
 'uri': '/Users/cjs14/GitHub/jupyter-cache/docs/using/example_nbs/basic.ipynb',
 'accessed': datetime.datetime(2020, 3, 13, 0, 2, 58, 780068)}

In [42]:
record = cache.get_staged_record(2)
print(record.traceback)

Traceback (most recent call last):
  File "/Users/cjs14/GitHub/jupyter-cache/jupyter_cache/executors/basic.py", line 152, in execute
    executenb(nb_bundle.nb, cwd=tmpdirname)
  File "/anaconda/envs/mistune/lib/python3.7/site-packages/nbconvert/preprocessors/execute.py", line 737, in executenb
    return ep.preprocess(nb, resources, km=km)[0]
  File "/anaconda/envs/mistune/lib/python3.7/site-packages/nbconvert/preprocessors/execute.py", line 405, in preprocess
    nb, resources = super(ExecutePreprocessor, self).preprocess(nb, resources)
  File "/anaconda/envs/mistune/lib/python3.7/site-packages/nbconvert/preprocessors/base.py", line 69, in preprocess
    nb.cells[index], resources = self.preprocess_cell(cell, resources, index)
  File "/anaconda/envs/mistune/lib/python3.7/site-packages/nbconvert/preprocessors/execute.py", line 448, in preprocess_cell
    raise CellExecutionError.from_cell_and_msg(cell, out)
nbconvert.preprocessors.execute.CellExecutionError: An error occurred while ex